In [1]:
import networkx as nx
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [2]:
data = pd.read_csv('data0.csv')
data["date"] = pd.to_datetime(data["date"]) 
date_begin=pd.to_datetime('2021-01-01')
date_end=pd.to_datetime('2022-12-31')
data=data[(data['date']>=date_begin)&(data['date']<=date_end)]

prediction=pd.read_csv('problem1_prediction.csv')
prediction['ds'] = pd.to_datetime(prediction['ds'])


date=pd.to_datetime('2023-01-01')
new_edge_value=1000
load_value=10000
res_value=100000
step=1

data.head()
# print(date+ pd.Timedelta(days=1))

,place1,place2,date,num
0,DC3,DC5,2021-01-01,3
1,DC3,DC10,2021-01-01,4
2,DC3,DC14,2021-01-01,4
3,DC5,DC3,2021-01-01,41
4,DC5,DC9,2021-01-01,3


In [3]:
def cost_func(u,v,not_used,current_flow,max_flow):
    load_rate=current_flow/max_flow
    res_flow=max_flow-current_flow
    if u=='s' or v=='t':
        return 0
    if '-pre' in u or '-next' in v:
        return 0
    return np.ceil(not_used*new_edge_value +load_rate*load_value+res_value/(res_flow+1))

In [4]:
unique_routes = data[['place1', 'place2']].drop_duplicates()
edge_info=pd.DataFrame(columns=['max_flow','place1','place2','current_flow'])
for index, row in unique_routes.iterrows():
    place1,place2=row['place1'],row['place2']
    if 'DC5' in [place1,place2]:
        continue
    max_flow_1=data[(data['place1']==place1) & (data['place2']==place2)]['num'].max()
    max_flow_2=prediction[(prediction['place1']==place1) & (prediction['place2']==place2)]['yhat'].max()
    max_flow=max(max_flow_1,max_flow_2)
    current_flow=prediction[(prediction['ds']==date) & (prediction['place1']==place1) & (prediction['place2']==place2)]['yhat'].values[0]
    edge_info.loc[index]=[max_flow,place1,place2,current_flow]
edge_info=edge_info.reset_index()
edge_info.head() 

,index,max_flow,place1,place2,current_flow
0,1,2011,DC3,DC10,1
1,2,242,DC3,DC14,0
2,8,1561,DC7,DC3,1
3,10,7759,DC7,DC9,0
4,11,6979,DC7,DC10,543


In [5]:
unique_points=pd.concat([data['place1'],data['place2']],ignore_index=True).drop_duplicates()
point_info=pd.DataFrame(columns=['place','res','max_flow','in_flow','out_flow'])
for index, value in unique_points.iteritems():
    if value=='DC5':
        continue
    in_flow=prediction[(prediction['ds']==date)&(prediction['place2']==value)&(prediction['place1']!='DC5') ]['yhat'].sum( )
    out_flow=prediction[(prediction['ds']==date)&(prediction['place1']==value)&(prediction['place2']!='DC5')]['yhat'].sum( )
    res=out_flow-in_flow
    
    max_in_sum_1_temp=data[(data['place2']==value)].groupby('date').sum()
    max_in_sum_1=max_in_sum_1_temp['num'].max()
    max_in_sum_2_temp=prediction[(prediction['place2']==value)][['yhat','ds']].groupby('ds').sum()
    max_in_sum_2=max_in_sum_2_temp['yhat'].max()
    if np.isnan(max_in_sum_1):
        max_in_sum_1=0
    if np.isnan(max_in_sum_2):
        max_in_sum_2=0
    max_in_sum=max(max_in_sum_1,max_in_sum_2)
    
    max_out_sum_1_temp=data[(data['place1']==value)].groupby('date').sum()
    max_out_sum_1=max_out_sum_1_temp['num'].max()
    max_out_sum_2_temp=prediction[(prediction['place1']==value)].groupby('ds').sum()
    max_out_sum_2=max_out_sum_2_temp['yhat'].max()
    if np.isnan(max_out_sum_1):
        max_out_sum_1=0
    if np.isnan(max_out_sum_2):
        max_out_sum_2=0
    max_out_sum=max(max_out_sum_1,max_out_sum_2)
    
    point_info.loc[index]=[value,res,max(max_in_sum,max_out_sum),in_flow,out_flow]
point_info=point_info.reset_index()
point_info.head()

,index,place,res,max_flow,in_flow,out_flow
0,0,DC3,-136,560123,141,5
1,8,DC7,3375,14128,0,3375
2,13,DC9,-50,1118681,62,12
3,21,DC10,-93412,1275839,210513,117101
4,28,DC12,-8,118782,15,7


In [6]:
# 创建有向图
G = nx.DiGraph()

# 添加节点
G.add_node('s')  # 源节点
G.add_node('t')  # 汇节点 

G.nodes['s']['demand']=-step
G.nodes['t']['demand']=step
for index, row in point_info.iterrows():
    point,max_flow,res,in_flow,out_flow=row['place'],row['max_flow'],row['res'],row['in_flow'],row['out_flow']
    G.add_node(point)
    G.add_node(point+'-pre')
    G.add_node(point+'-next')
    
    G.add_edge(point+'-pre', point, capacity=max(max_flow-in_flow,0), weight=0,flow=0,origin_capacity=max(0,max_flow-in_flow))
    G.add_edge(point, point+'-next', capacity=max(max_flow-out_flow,0), weight=0,flow=0,origin_capacity=max(0,max_flow-in_flow))
# print(G.nodes())
# 添加边
for index, row in edge_info.iterrows():
    u,v,max_flow,current_flow=row['place1'],row['place2'],row['max_flow'],row['current_flow']
    cost=cost_func(u,v,True,current_flow,max_flow)
    G.add_edge(u+'-next', v+'-pre', capacity=max(0,max_flow-current_flow), weight=cost,flow=current_flow,origin_capacity=max_flow)

for index, row in point_info.iterrows():
    place,res,max_flow=row['place'],row['res'],row['max_flow']
    if res>0:
        G.add_edge('s', place+'-pre', capacity=1e8, weight=0,flow=0,origin_capacity=1e8)
    else:
        G.add_edge(place+'-next', 't', capacity=1e8, weight=0,flow=0,origin_capacity=1e8)


flow_to_DC5=prediction[(prediction['ds']==date) & (prediction['place2']=='DC5')]['yhat'].sum()
flowDict=nx.max_flow_min_cost(G, 's', 't', capacity='capacity', weight='weight')
max_flow=sum(flowDict[node1][node2] for node1 in flowDict for node2 in flowDict[node1])
print(flow_to_DC5,max_flow)
min_cost_list=[]
flow_dict_list=[]

add_num,i=1,0
while i<flow_to_DC5:
    print(f'{i}/{flow_to_DC5}')
    i+=add_num
    if add_num<step:
        add_num+=1
    flowDict = nx.min_cost_flow(G, demand="demand", capacity='capacity', weight='weight')
    min_cost=nx.cost_of_flow(G,flowDict)
    min_cost_list.append(min_cost)
    flow_dict_list.append(flowDict)
    # print(min_cost)
    for u, v in G.edges():
        if flowDict[u][v]!=0:
            # print(u,v,flowDict[u][v])
            G[u][v]['capacity']-=flowDict[u][v]
            G[u][v]['flow']  += flowDict[u][v]
            G[u][v]['weight']=cost_func(u,v,False,G[u][v]['flow'],G[u][v]['origin_capacity'])
            # print(u,v,G[u][v]['weight'])
    


115223 27964237
0/115223
1/115223
2/115223
3/115223
4/115223
5/115223
6/115223
7/115223
8/115223
9/115223
10/115223
11/115223
12/115223
13/115223
14/115223
15/115223
16/115223
17/115223
18/115223
19/115223
20/115223
21/115223
22/115223
23/115223
24/115223
25/115223
26/115223
27/115223
28/115223
29/115223
30/115223
31/115223
32/115223
33/115223
34/115223
35/115223
36/115223
37/115223
38/115223
39/115223
40/115223
41/115223
42/115223
43/115223
44/115223
45/115223
46/115223
47/115223
48/115223
49/115223
50/115223
51/115223
52/115223
53/115223
54/115223
55/115223
56/115223
57/115223
58/115223
59/115223
60/115223
61/115223
62/115223
63/115223
64/115223
65/115223
66/115223
67/115223
68/115223
69/115223
70/115223
71/115223
72/115223
73/115223
74/115223
75/115223
76/115223
77/115223
78/115223
79/115223
80/115223
81/115223
82/115223
83/115223
84/115223
85/115223
86/115223
87/115223
88/115223
89/115223
90/115223
91/115223
92/115223
93/115223
94/115223
95/115223
96/115223
97/115223
98/115223
99/1

In [7]:
result_dict = {}
for d in flow_dict_list:
    for k1, v1 in d.items():
        if k1 not in result_dict:
            result_dict[k1] = {}
        for k2, v2 in v1.items():
            result_dict[k1][k2] = result_dict[k1].get(k2, 0) + v2
result_dict_new={}          
for k1,v1 in result_dict.items():
    for k2,v2 in v1.items():
        if v2!=0:
            k1=k1.replace('-pre','').replace('-next','')
            k2=k2.replace('-pre','').replace('-next','')
            if k1==k2:
                continue
            if k1 not in result_dict_new:
                result_dict_new[k1] = {}
            result_dict_new[k1][k2] = v2
result_dict=result_dict_new
for k1,v1 in result_dict.items():
    for k2,v2 in v1.items():
        if not(k1=='s' or k2=='t'):
            max_flow=edge_info[(edge_info['place1']==k1) & (edge_info['place2']==k2)]['max_flow'].values[0]
            origin_flow=edge_info[(edge_info['place1']==k1) & (edge_info['place2']==k2)]['current_flow'].values[0]
            result_dict[k1][k2] = [v2,(v2+origin_flow)/max_flow,max_flow,origin_flow]
        else:
            result_dict[k1][k2] = [v2,0,0,0]


df = pd.DataFrame([(k1, k2, v2[0],v2[1],v2[2],v2[3],date) for k1,v1 in result_dict.items() for k2,v2 in v1.items()], columns=['place1', 'place2', 'flow','load_rate','max_flow','origin_flow','date'])
df.to_csv('problem2_01.csv', index=True)  
result_dict
            

{'s': {'DC15': [13041, 0, 0, 0],
  'DC17': [5000, 0, 0, 0],
  'DC19': [5476, 0, 0, 0],
  'DC23': [5235, 0, 0, 0],
  'DC25': [17711, 0, 0, 0],
  'DC36': [6089, 0, 0, 0],
  'DC44': [12460, 0, 0, 0],
  'DC47': [12228, 0, 0, 0],
  'DC48': [6300, 0, 0, 0],
  'DC51': [31168, 0, 0, 0],
  'DC58': [515, 0, 0, 0]},
 'DC3': {'t': [21292, 0, 0, 0]},
 'DC9': {'t': [65603, 0, 0, 0]},
 'DC10': {'t': [515, 0, 0, 0]},
 'DC12': {'t': [12591, 0, 0, 0]},
 'DC15': {'DC9': [8025, 0.10006983066064792, 80194, 0],
  'DC4': [5016, 0.0999940194964416, 50163, 0]},
 'DC17': {'DC3': [5000, 0.09998400255959046, 50008, 0]},
 'DC19': {'DC12': [5476, 0.09999817388287285, 54761, 0]},
 'DC23': {'DC3': [5235, 0.09999044981377136, 52355, 0]},
 'DC25': {'DC9': [12618, 0.10011345874624118, 126037, 0],
  'DC4': [5093, 0.09999607319563339, 50932, 0]},
 'DC36': {'DC3': [6089, 0.10003285690816494, 60870, 0]},
 'DC44': {'DC9': [12460, 0.10011329032050716, 124459, 0]},
 'DC47': {'DC12': [7115, 0.1000534368320396, 71112, 0],
  'DC7

In [8]:
# G_result = nx.DiGraph()

# # 添加边和边的属性
# for u in result_dict:
#     for v, flow in result_dict[u].items():
#         if result_dict[u][v]==0:
#             continue
#         if u=='s' or v=='t':
#             continue
#         G_result.add_node(u)
#         G_result.add_node(v)
#         G_result.add_edge(u, v, flow=flow, capacity=G[u][v]['capacity'], weight=G[u][v]['weight'])

# # 可视化
# pos = nx.spring_layout(G_result,k=10)
# plt.figure(figsize=(8, 8))
# nx.draw_networkx_nodes(G_result, pos, node_size=50)
# nx.draw_networkx_labels(G_result, pos, font_size=8)
# edge_labels = {(u, v): f"{d['flow']}/{d['capacity']}" for u, v, d in G_result.edges(data=True)}
# nx.draw_networkx_edges(G_result, pos, alpha=0.8)
# nx.draw_networkx_edge_labels(G_result, pos, edge_labels=edge_labels, font_size=6)
# plt.axis('off')
# plt.show()